# Task 4: build a character based language model

## Characters and vocabulary

The main expected advantage of working with characters and no longer with words as tokens to build a language model is to drastically reduce the number of classes given to the multiclass classifier.

Most tutorials on character based language model assume that the list of unique tokens or characters is limited to ascii characters. Basically the alphabet, uppercase, lowercase, the digits and some punctuation.

However as we will see here this is not the case with real world, user content generated corpuses.

So let's start by and listing all the unique characters we find in our raw dataset.






In [2]:
import pandas as pd
import numpy as np

In [2]:
# load the dataset
df_raw = pd.read_csv('../Dataset/stackexchange_812k1.tokenized.csv').sample(frac = 1, random_state = 8).reset_index(drop = True)

Let's now concatenate all the original texts from the dataset and list the unique characters. There are 746 of them!

In [3]:
text = ''.join(df_raw.text.values).lower()

# simple but efficient way to split string into list of characters
all_characters = [s for s in text]
unique_characters = np.unique(all_characters) 
print(unique_characters)


['\t' '\x0b' '\x0c' ' ' '!' "'" ',' '-' '.' '?' '\\' 'a' 'b' 'c' 'd' 'e'
 'f' 'g' 'h' 'i' 'j' 'k' 'l' 'm' 'n' 'o' 'p' 'q' 'r' 's' 't' 'u' 'v' 'w'
 'x' 'y' 'z' '\x7f' '\xa0' '¡' '¢' '£' '¥' '¦' '§' '¨' '©' 'ª' '«' '¬'
 '\xad' '®' '¯' '°' '±' '²' '³' '´' 'µ' '¶' '·' '¹' 'º' '»' '¼' '½' '¾'
 '¿' '×' 'ß' 'à' 'á' 'â' 'ã' 'ä' 'å' 'æ' 'ç' 'è' 'é' 'ê' 'ë' 'ì' 'í' 'î'
 'ï' 'ð' 'ñ' 'ò' 'ó' 'ô' 'õ' 'ö' '÷' 'ø' 'ù' 'ú' 'û' 'ü' 'ý' 'ā' 'ă' 'ą'
 'ć' 'č' 'ē' 'ĕ' 'ė' 'ę' 'ğ' 'ī' 'ı' 'ĺ' 'ļ' 'ł' 'ń' 'ō' 'ő' 'œ' 'ř' 'ś'
 'ş' 'š' 'ū' 'ů' 'ŷ' 'ź' 'ž' 'ƒ' 'ơ' 'ƴ' 'ț' 'ȳ' 'ɑ' 'ə' 'ɛ' 'ɣ' 'ɪ' 'ɵ'
 'ʃ' 'ʊ' 'ʒ' 'ʼ' 'ˆ' 'ˇ' 'ˈ' 'ˉ' 'ˌ' '˙' '˚' '˜' '̀' '́' '̂' '̃' '̄' '̅'
 '̇' '̈' '̧' '̶' '̸' '͝' ';' '΄' 'ά' 'έ' 'ή' 'ί' 'α' 'β' 'γ' 'δ' 'ε' 'ζ'
 'η' 'θ' 'ι' 'κ' 'λ' 'μ' 'ν' 'ξ' 'ο' 'π' 'ρ' 'ς' 'σ' 'τ' 'υ' 'φ' 'χ' 'ψ'
 'ω' 'ό' 'ύ' 'ώ' 'ϐ' 'ϕ' 'ϵ' 'а' 'б' 'в' 'г' 'д' 'е' 'ж' 'з' 'и' 'й' 'к'
 'л' 'м' 'н' 'о' 'п' 'р' 'с' 'т' 'у' 'ф' 'х' 'ц' 'ч' 'ш' 'щ' 'ъ' 'ы' 'ь'
 'э' 'ю' 'я' 'ё' 'є' 'א' 'ב' 'ד' 'ה' 'ו' 'ח' 'י' 'כ' '

As you can see, when you list all the unique characters that exist in the original text, the list is not limited to letters, digits and common punctuations signs.

We end up with many different types of characters:

* greek alphabet: 'α', 'β', 'γ', ...
* kanji signs, hebrew, arabic and cyrillic letters: '帰', '形', '松', 'ц', 'ч', 'ш', 'щ', 'ب', 'ة', 'ت', 'א', 'ב', 'ד', 'ה', ...
* emojis: '🔥', '😀', '😁', ...
* mathematical symbols: '∑', 'ℝ', '≫', '≰', '𝒩', '𝔼', ...
* UTF-16 and other encodings: '\U0010fc03', '\uf0de', ...
* other: '®', '†', '‡', '•', ...


We could either take into account all the different signs in the dataset but then we would lose the advantage given by considering characters insetad of words. Although the number of target classes is new less than a thousand, down from a vocabulary size of several thousands, we still have a large target space.

So instead we will arbitrarily choose a subset of all characters as the valid ones and discard the other ones as invalid or unknowns.

Note also that the list of unique characters does not contain digits. Which is weird.

So let's define our vocabulary by keeping the N most common characters in our dataset.


In [4]:
from collections import Counter

char_count = Counter(all_characters)
print(char_count.most_common())

# limit the allowed characters to MAX_VOCAB_SIZE
MAX_VOCAB_SIZE = 40
valid_characters = [t[0] for t in  char_count.most_common(MAX_VOCAB_SIZE)]
valid_characters.sort()


[(' ', 41838479), ('e', 22649492), ('t', 18848027), ('a', 15614041), ('i', 15307965), ('o', 14812551), ('s', 13076523), ('n', 12756881), ('r', 11311575), ('h', 8004404), ('l', 7935954), ('d', 6606091), ('u', 6013733), ('c', 5874121), ('m', 5307240), ('f', 4206025), ('p', 4142895), ('y', 3538913), ('g', 3390453), ('w', 2959613), ('b', 2934463), ('.', 2818634), ('v', 2193287), (',', 2137165), ('k', 1184558), ('-', 949680), ('x', 721059), ("'", 627955), ('?', 430168), ('q', 420156), ('z', 216752), ('j', 214419), ('!', 65816), ('\\', 48970), ('’', 17695), ('‘', 9226), ('–', 5094), ('═', 2808), ('—', 1513), ('β', 1284), ('−', 1073), ('é', 957), ('σ', 894), ('…', 836), ('´', 827), ('θ', 801), ('μ', 715), ('α', 704), ('║', 657), ('²', 600), ('─', 507), ('λ', 405), ('\xa0', 371), ('×', 321), ('±', 321), ('°', 307), ('•', 307), ('\u200b', 298), ('ö', 296), ('≤', 239), ('ü', 237), ('ε', 231), ('δ', 202), ('£', 195), ('π', 184), ('\xad', 184), ('\t', 180), ('γ', 173), ('µ', 169), ('∗', 169), ('ﬁ'

There's a balance to be found between keeping as many characters as possible to reflect the diversity of the corpus and limiting the number of classes to allow the model to reach satisfactory scores.


# Training dataset

As usual since we have limited computational resources, we want to limit the volume of the training dataset.

Here we will simply sample N titles from the original dataset. Titles are usually composed of one sentence and are shorter than posts or comments. 
Titles are also composed of more classic character tokens. 


In [5]:
POSTS_TYPE = 'title'
DF_SAMPLE_COUNT = 10000

In [6]:
# subsample the original dataset

df = df_raw[(df_raw.category == POSTS_TYPE)].sample(DF_SAMPLE_COUNT).reset_index(drop = True)

print("df.shape: ", df.shape)

print(df.text.sample(2).values)


df.shape:  (10000, 7)
['How could I visualise the importance of different inputs to the forecast for a black-box non-linear model?'
 'Tail inequality on sum of product of normal variables']


# AllenNLP code structure

The structure of the AllenNLP framework is tailored for NLP tasks. 
It is composed of the following parts:

### Instances 
An Instance is an abstraction of a training example composed of the input objects and the output objects. In our case, the input is a sequence of tokens (characters) and the output the token (character) following the sequence.

The Instance also includes the tokens indexers.

### Model

A Model is a subclass of torch.nn.Module. It needs a forward method that takes tensor inputs and produces a dict of tensor outputs that includes the loss used to train the model. 

Our model will consist of 

* an embedding layer, 
* a sequence to sequence encoder, LSTM 
* and a feedforward network that outputs the probability distribution over the vocabulary.

We'll add a text generation method.

Note: The AllenNLP code in this notebook is taken from the notebook 
https://colab.research.google.com/github/mhagiwara/realworldnlp/blob/master/examples/generation/lm.ipynb
accompanying the book Real World NLP by Masato Hagiwara


## Install and Packages

Let's install AllenNLP and load the necessary librairies

Import the libs and load the data

In [7]:
import re
from typing import Dict, List, Tuple, Set

import torch
import torch.optim as optim
from allennlp.common.file_utils import cached_path
from allennlp.common.util import START_SYMBOL, END_SYMBOL
from allennlp.data.fields import TextField
from allennlp.data.instance import Instance
from allennlp.data.iterators import BasicIterator
from allennlp.data.token_indexers import TokenIndexer, SingleIdTokenIndexer
from allennlp.data.tokenizers import Token, CharacterTokenizer
from allennlp.data.vocabulary import Vocabulary, DEFAULT_PADDING_TOKEN
from allennlp.models import Model
from allennlp.modules.seq2seq_encoders import PytorchSeq2SeqWrapper
from allennlp.modules.text_field_embedders import TextFieldEmbedder, BasicTextFieldEmbedder
from allennlp.modules.token_embedders import Embedding
from allennlp.nn.util import get_text_field_mask, sequence_cross_entropy_with_logits
from allennlp.training.trainer import Trainer


# Tokenizer and Instances

The AllenNLP CharacterTokenizer is tailored to the framework and we will use it insetad of simply splitting the sentences into list of charaters in simple python.





In [8]:
tokenizer = CharacterTokenizer()

The train set is composed of tokenized sequences of the original text

In [9]:
train_set = df.text.apply(lambda txt : tokenizer.tokenize(txt.lower())).values

Finally let's define the function that will generate an Instance for each list of token. The function takes a list of tokens and an indexer as input and returns an instance composed of the input and output tokens.

In [10]:

def tokens_to_instance(tokens: List[Token], token_indexers: Dict[str, TokenIndexer]):
    tokens = list(tokens)
    tokens.insert(0, Token(START_SYMBOL))
    tokens.append(Token(END_SYMBOL))

    input_field  = TextField(tokens[:-1], token_indexers)
    output_field = TextField(tokens[1:], token_indexers)
    return Instance({'input_tokens': input_field, 'output_tokens': output_field})        


In the previous task we had to construct the mapping between the tokens and their unique index in the vocabulary. The SingleIdTokenIndexer takes care of that.


In [11]:

token_indexers = {'tokens': SingleIdTokenIndexer()}
instances = [tokens_to_instance(tokens, token_indexers) for tokens in train_set]

## Vocabulary
The Vocabulary object takes a dict of tokens and their counts. In our case the counts can all be set to 1.


In [12]:
token_counts = {char: 1 for char in valid_characters}
vocab = Vocabulary({'tokens': token_counts})


# Model

Let's create the model and set some variables.

In [13]:
EMBEDDING_SIZE = 32
HIDDEN_SIZE = 256
BATCH_SIZE = 128

In [438]:
class RNNLanguageModel(Model):
    def __init__(self,
                 embedder: TextFieldEmbedder,
                 hidden_size: int,
                 max_len: int,
                 vocab: Vocabulary) -> None:
        super().__init__(vocab)

        self.embedder = embedder

        # initialize a Seq2Seq encoder, LSTM
        self.rnn = PytorchSeq2SeqWrapper(
            torch.nn.LSTM(EMBEDDING_SIZE, HIDDEN_SIZE, batch_first=True))

        self.hidden2out = torch.nn.Linear(in_features=self.rnn.get_output_dim(), out_features=vocab.get_vocab_size('tokens'))
        self.hidden_size = hidden_size
        self.max_len = max_len
        

    def forward(self, input_tokens, output_tokens):
        '''
        This is the main process of the Model where the actual computation happens. 
        Each Instance is fed to the forward method. 
        It takes dicts of tensors as input, with same keys as the fields in your Instance (input_tokens, output_tokens)
        It outputs the results of predicted tokens and the evaluation metrics as a dictionary. 
        '''

        mask = get_text_field_mask(input_tokens)
        embeddings = self.embedder(input_tokens)
        rnn_hidden = self.rnn(embeddings, mask)
        out_logits = self.hidden2out(rnn_hidden)
        
        loss = sequence_cross_entropy_with_logits(out_logits, output_tokens['tokens'], mask)
        
        #tensor = out_logits[:,out_logits.size()-1]
        #state = (torch.zeros(1, 1, self.hidden_size), torch.zeros(1, 1, self.hidden_size))
        words=[]
       
        output_t=out_logits[:,out_logits.size()[1]-1]
        
        test=torch.argmax(output_t).item()
        #if torch.argmax(output_t).item().size()[1]:
        try:
          token = Token(text=self.vocab.get_token_from_index(test, 'tokens'))
          words.append(token)
        except:
          print("An exception occurred could be ignored") 
       
        return {'loss': loss, 'nextCaracter': words}
    
    def generateN(self) -> Tuple[List[Token], torch.tensor]:
    
        start_symbol_idx = self.vocab.get_token_index(START_SYMBOL, 'tokens')
        end_symbol_idx = self.vocab.get_token_index(END_SYMBOL, 'tokens')
        padding_symbol_idx = self.vocab.get_token_index(DEFAULT_PADDING_TOKEN, 'tokens')

        log_likelihood = 0.
        words = []
        state = (torch.zeros(1, 1, self.hidden_size), torch.zeros(1, 1, self.hidden_size))

        word_idx = start_symbol_idx

       
        tokens = torch.tensor([[word_idx]])

        embeddings = self.embedder({'tokens': tokens})
        output, state = self.rnn._module(embeddings, state)
        output = self.hidden2out(output)
        
        log_prob = torch.log_softmax(output[0, 0], dim=0)
        dist = torch.exp(log_prob)
        #log_prob = torch.log_softmax(tensor)
        word_idx = torch.multinomial(
                    dist, num_samples=1, replacement=False).item()
        token = Token(text=self.vocab.get_token_from_index(word_idx, 'tokens'))
        
        return {'loss': log_prob}
        
    
    def generateM(self) -> Tuple[List[Token], torch.tensor]:

        start_symbol_idx = self.vocab.get_token_index(START_SYMBOL, 'tokens')
        end_symbol_idx = self.vocab.get_token_index(END_SYMBOL, 'tokens')
        padding_symbol_idx = self.vocab.get_token_index(DEFAULT_PADDING_TOKEN, 'tokens')

        log_likelihood = 0.
        words = []
        state = (torch.zeros(1, 1, self.hidden_size), torch.zeros(1, 1, self.hidden_size))

        word_idx = start_symbol_idx

       
        tokens = torch.tensor([[word_idx]])

        embeddings = self.embedder({'tokens': tokens})
        output, state = self.rnn._module(embeddings, state)
        output = self.hidden2out(output)
        
        log_prob = torch.log_softmax(output[0, 0], dim=0)

            #dist = torch.exp(log_prob)

            #word_idx = start_symbol_idx

           
        return {'loss': log_prob}

    def generate(self) -> Tuple[List[Token], torch.tensor]:

        start_symbol_idx = self.vocab.get_token_index(START_SYMBOL, 'tokens')
        end_symbol_idx = self.vocab.get_token_index(END_SYMBOL, 'tokens')
        padding_symbol_idx = self.vocab.get_token_index(DEFAULT_PADDING_TOKEN, 'tokens')

        log_likelihood = 0.
        words = []
        state = (torch.zeros(1, 1, self.hidden_size), torch.zeros(1, 1, self.hidden_size))

        word_idx = start_symbol_idx

        for i in range(self.max_len):
            tokens = torch.tensor([[word_idx]])

            embeddings = self.embedder({'tokens': tokens})
            output, state = self.rnn._module(embeddings, state)
            output = self.hidden2out(output)

            log_prob = torch.log_softmax(output[0, 0], dim=0)

            dist = torch.exp(log_prob)

            word_idx = start_symbol_idx

            while word_idx in {start_symbol_idx, padding_symbol_idx}:
                word_idx = torch.multinomial(
                    dist, num_samples=1, replacement=False).item()

            log_likelihood += log_prob[word_idx]

            if word_idx == end_symbol_idx:
                break

            token = Token(text=self.vocab.get_token_from_index(word_idx, 'tokens'))
            words.append(token)

        return words, log_likelihood

Instantiate 
* the embeddings of size EMBEDDING_SIZE
* the RNN model
* the optimizer and batch processing with BATCH_SIZE
* the Trainer

And train the model!

# NOTE

Training the model takes a while (this one took 4mn per epoch). The google Colab environment will loose connection.

In that case, you simply have to reconnect and run the cell for the computation to continue from where it stopped. No need to start the notebook from cratch as the kernel is not restarted by a disconnect. At least not right away.

This also implies that it's best to keep an eye on the process to avoid having to restart from the beginning.

In [439]:
token_embedding = Embedding(num_embeddings=vocab.get_vocab_size('tokens'), embedding_dim=EMBEDDING_SIZE)

embedder = BasicTextFieldEmbedder({"tokens": token_embedding})

model = RNNLanguageModel(embedder=embedder, hidden_size=HIDDEN_SIZE, max_len=80, vocab=vocab)

In [440]:
iterator = BasicIterator(batch_size=BATCH_SIZE)
iterator.index_with(vocab)

optimizer = optim.Adam(model.parameters(), lr=5.e-3)

In [446]:
trainer = Trainer(model=model,
                  optimizer=optimizer,
                  iterator=iterator,
                  train_dataset=instances,
                  num_epochs=20)
trainer.train()

  0%|                                                                                           | 0/79 [00:00<?, ?it/s]

An exception occurred


loss: 2.9129 ||:   1%|▊                                                                 | 1/79 [00:05<07:04,  5.45s/it]

An exception occurred


loss: 2.9076 ||:   3%|█▋                                                                | 2/79 [00:12<07:27,  5.81s/it]

...

An exception occurred


loss: 1.0221 ||:  99%|████████████████████████████████████████████████████████████████▏| 78/79 [07:26<00:05,  5.39s/it]

An exception occurred


loss: 1.0230 ||: 100%|█████████████████████████████████████████████████████████████████| 79/79 [07:26<00:00,  5.66s/it]


{'best_epoch': 9,
 'peak_cpu_memory_MB': 0,
 'training_duration': '1:13:36.336172',
 'training_start_epoch': 0,
 'training_epochs': 9,
 'epoch': 9,
 'training_loss': 1.0230346827567378,
 'training_cpu_memory_MB': 0.0}

In [347]:
def predict(text: str, model: Model) -> float:
    tokenizer = CharacterTokenizer()
    tokens = tokenizer.tokenize(text)
    
    token_indexers = {'tokens': SingleIdTokenIndexer()}
    instance = tokens_to_instance(tokens, token_indexers)
    output = model.forward_on_instance(instance)
    print(output)

Instead of calculating the perplexity of sentences, let's see which loss is associated to the sentence.

In [460]:


sentence = "distribution"


for _ in range(100):
  t=predict(sentence, model)

  sentence=sentence+t['nextCaracter'].text
print(sentence)






distribution of the regression model with a continuous variables@@UNKNOWN@@redect statistical test in r@@UNKNOWN@@repect the same da


The sentence with words such as "Southern California" and "San Diego way"  scores worse than the sentence that makes no grammatical sense.

These words are not found in the training dataset and the model has had no opportunity to learn about their associated character sequences.

## Text generation


In [136]:
output = model.generateN()
print(output)

{'loss': tensor([-3.6887, -3.8121, -3.6887, -3.7709, -3.6877, -3.7277, -3.7258, -3.7607,
        -3.7670, -3.7773, -3.7353, -3.8029, -3.7438, -3.7261, -3.7671, -3.7849,
        -3.6975, -3.7456, -3.7873, -3.6754, -3.6870, -3.7058, -3.7248, -3.7133,
        -3.7031, -3.7003, -3.7902, -3.6809, -3.7846, -3.7556, -3.7391, -3.7068,
        -3.7770, -3.7845, -3.7832, -3.7101, -3.7536, -3.7359, -3.7083, -3.6868,
        -3.7837, -3.7256], grad_fn=<LogSoftmaxBackward>)}


In [1]:
for _ in range(50):
    tokens, _ = model.generate()
    print(''.join(token.text for token in tokens))

NameError: name 'model' is not defined

# Conclusion

The character base language model ends up producing better text than the word based LM of the last task.

This may be due to a bad performance of the word based language model.

However the character based LM ouptuts are really impressive.

